In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kKZSBpgDwRCvaMR9y9caEs1wSdbFKRzs' -O QA.csv # Downloading qa.csv from google drive
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/summarization/run_summarization.py -O run_summarization.py 

# Installing the libraries that run_summarization.py uses
!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install rouge_score

--2021-07-06 05:32:23--  https://docs.google.com/uc?export=download&id=1kKZSBpgDwRCvaMR9y9caEs1wSdbFKRzs
Resolving docs.google.com (docs.google.com)... 142.251.33.206, 2607:f8b0:4004:837::200e
Connecting to docs.google.com (docs.google.com)|142.251.33.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tdfn1caq8950erb092r9i2pa5img6skt/1625549550000/03976892977300334194/*/1kKZSBpgDwRCvaMR9y9caEs1wSdbFKRzs?e=download [following]
--2021-07-06 05:32:32--  https://doc-0s-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tdfn1caq8950erb092r9i2pa5img6skt/1625549550000/03976892977300334194/*/1kKZSBpgDwRCvaMR9y9caEs1wSdbFKRzs?e=download
Resolving doc-0s-bk-docs.googleusercontent.com (doc-0s-bk-docs.googleusercontent.com)... 142.250.188.193, 2607:f8b0:4004:836::2001
Connecting to doc-0s-bk-docs.googleusercontent.com (doc-0s-bk-docs.goog

In [ ]:
import pandas as pd
import math
from google.colab import files

In [ ]:
qa = pd.read_csv("QA.csv")
qa = qa.dropna()

# Creates a split so that we have 60% training and 40% validation
split = math.floor(len(qa.index)*0.6)

# Put qa.csv data into required format for run_summarization.py
text_column = []
summary_column = []
for i in range(len(qa.index)):
  text_column.append(qa["context"].values[i]+"<q>"+qa["question"].values[i])
  summary_column.append(str(qa["answer"].values[i])+"")

train = {"text_column":text_column[0:split], "summary_column":summary_column[0:split]}
train = pd.DataFrame.from_dict(train)
train.to_csv("qa_bart_train_format.csv", index=False)

eval = {"text_column":text_column[split:], "summary_column":summary_column[split:]}
eval = pd.DataFrame.from_dict(eval)
eval.to_csv("qa_bart_eval_format.csv", index=False)

# files.download("qa_bart_train_format.csv")
# files.download("qa_bart_eval_format.csv")

In [ ]:
!python run_summarization.py \
    --model_name_or_path facebook/bart-base \
    --do_train \
    --do_eval \
    --train_file qa_bart_train_format.csv \
    --validation_file qa_bart_eval_format.csv \
    --source_prefix "summarize: " \
    --output_dir /tmp/tst-summarization \
    --overwrite_output_dir \
    --save_total_limit=5\
    --per_device_train_batch_size=2 \
    --per_device_eval_batch_size=2 \
    --predict_with_generate \
    --text_column text_column \
    --summary_column summary_column

2021-07-04 21:33:39.380667: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
07/04/2021 21:33:40 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/04/2021 21:33:40 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
le

In [ ]:
!zip -r bart.zip /tmp/tst-summarization/checkpoint-78500

	zip warning: name not matched: /dpr

zip error: Nothing to do! (try: zip -r dpr.zip . -i /dpr)


In [ ]:
!pip install --upgrade gupload

from google.colab import auth

# Authenticate and create the PyDrive client.
auth.authenticate_user()

!gupload --to '1W9R77oTk_DAMxfSjKGOLNj5xFtoC7gLF' bart.zip

In [ ]:
import torch
print(torch.__version__)

1.9.0+cu102
